In [33]:
from github import Github,Auth
from IPython.display import HTML
from decouple import config
from jinja2 import Environment, FileSystemLoader


In [34]:
g = Github(auth=Auth.Token(config("GITHUB_TOKEN")))

In [35]:
g.get_user().name

'Giorgio Caizzi'

In [36]:
repos = [repo for repo in g.get_user().get_repos() if repo.name in ["pysurfline","py3dinterpolations","repomanager"]]

In [37]:
len(repos)

3

In [38]:
class Repository:
    def __init__(self,name,description,url,language):
        self.name = name
        self.description = description
        self.url = url
        self.language = language

    def __repr__(self):
        return f"Repository({self.name},{self.description},{self.url},{self.language})"


In [39]:
repos = [
    Repository(
            name= repo.name,
            language=repo.language,
            description=repo.description,
            url=repo.url,
    ) for repo in repos
]

In [40]:
repos

[Repository(py3dinterpolations,None,https://api.github.com/repos/giocaizzi/py3dinterpolations,Python),
 Repository(pysurfline,Python Surfline API,https://api.github.com/repos/giocaizzi/pysurfline,Python),
 Repository(repomanager,None,https://api.github.com/repos/giocaizzi/repomanager,Jupyter Notebook)]

In [41]:
# Set up the environment
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('template.html')

In [42]:
html = template.render(repos=repos)

In [43]:
HTML(html)

Name,py3dinterpolations,pysurfline,repomanager
Description,None,Python Surfline API,None
Language,Python,Python,Jupyter Notebook
URL,https://api.github.com/repos/giocaizzi/py3dinterpolations,https://api.github.com/repos/giocaizzi/pysurfline,https://api.github.com/repos/giocaizzi/repomanager


In [44]:
with open('index.html', 'w') as file:
    file.write(html)